In [ ]:
````xml
<VSCode.Cell language="markdown">
# Coleta DIEESE - Cesta Básica Nacional (Versão Funcional)

**Projeto:** Top One Model - Sistema de Modelagem de Risco de Crédito

**Objetivo:** Automatizar coleta de dados da cesta básica do DIEESE seguindo o fluxo correto do site

**Fluxo do Site DIEESE:**
1. **Página inicial:** https://www.dieese.org.br/cesta/
   - Preencher campos: `dataInicial` e `dataFinal` (formato mmaaaa)
   - Clicar botão "Consulta"

2. **Página de resultados:** https://www.dieese.org.br/cesta/produto
   - Visualizar tabela com dados
   - Clicar botão "Exporta Excel" para download

**Estrutura de Saída:**
```
data/external_data/macro_specified_data/cesta_basica/
├── raw_data/           # Planilhas Excel do DIEESE
├── processed_data/     # Dados processados
└── metadata/          # Logs de coleta
```
</VSCode.Cell>

<VSCode.Cell language="python">
# Configuração de caminhos (mantém estrutura existente)
from pathlib import Path
from datetime import datetime
import sys, subprocess, os
import pandas as pd

project_root = Path('/home/usuario/Documentos/top_one_model_01')
cesta_basica_root = project_root / 'data' / 'external_data' / 'macro_specified_data' / 'cesta_basica'
raw_data_path = cesta_basica_root / 'raw_data'
processed_data_path = cesta_basica_root / 'processed_data'
metadata_path = cesta_basica_root / 'metadata'

# Criar estrutura de pastas
for p in (raw_data_path, processed_data_path, metadata_path):
    p.mkdir(parents=True, exist_ok=True)

print('✅ Estrutura de pastas configurada:')
print(f'   📂 Raw data: {raw_data_path}')
print(f'   📂 Processed: {processed_data_path}')
print(f'   📂 Metadata: {metadata_path}')
</VSCode.Cell>

<VSCode.Cell language="python">
# Instalar e configurar Playwright (Chromium empacotado)
import importlib

def setup_playwright():
    """Instala Playwright e Chromium se necessário"""
    try:
        importlib.import_module('playwright')
        print('✅ Playwright já instalado')
    except ImportError:
        print('📦 Instalando Playwright...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'playwright'])
        print('✅ Playwright instalado')
    
    # Instalar navegador Chromium
    try:
        print('📦 Instalando Chromium...')
        subprocess.check_call([sys.executable, '-m', 'playwright', 'install', 'chromium'])
        print('✅ Chromium instalado com sucesso')
    except Exception as e:
        print(f'❌ Erro ao instalar Chromium: {e}')
        raise

setup_playwright()
print('🎯 Playwright configurado e pronto!')
</VSCode.Cell>

<VSCode.Cell language="python">
# Função principal de coleta seguindo o fluxo correto do DIEESE
from typing import Optional

def coletar_cesta_basica_dieese(data_inicial: str, data_final: str, timeout_ms: int = 30000) -> Optional[Path]:
    """
    Coleta dados da cesta básica do DIEESE seguindo o fluxo correto:
    1. Acessa https://www.dieese.org.br/cesta/
    2. Preenche campos dataInicial e dataFinal (formato mmaaaa)
    3. Clica "Consulta" -> redireciona para /cesta/produto
    4. Clica "Exporta Excel" para download
    
    Args:
        data_inicial: formato mmaaaa (ex: '012018')
        data_final: formato mmaaaa (ex: '122025') 
        timeout_ms: timeout em milissegundos
        
    Returns:
        Path do arquivo baixado ou None se falhou
    """
    from playwright.sync_api import sync_playwright
    
    print(f'🎯 Coletando dados DIEESE: {data_inicial} → {data_final}')
    
    with sync_playwright() as p:
        # Configurar navegador
        browser = p.chromium.launch(headless=True)
        context = browser.new_context(accept_downloads=True)
        page = context.new_page()
        page.set_default_timeout(timeout_ms)
        
        try:
            # PASSO 1: Acessar página inicial do DIEESE
            print('   📡 Acessando https://www.dieese.org.br/cesta/')
            page.goto('https://www.dieese.org.br/cesta/', wait_until='domcontentloaded')
            page.wait_for_load_state('networkidle')
            
            # PASSO 2: Preencher campos de data
            print(f'   📝 Preenchendo datas: {data_inicial} - {data_final}')
            
            # Localizar e preencher campo data inicial
            page.fill('input[name="dataInicial"]', data_inicial)
            
            # Localizar e preencher campo data final  
            page.fill('input[name="dataFinal"]', data_final)
            
            # PASSO 3: Clicar botão Consulta
            print('   🔍 Clicando botão "Consulta"')
            page.click('input[type="submit"][value="Consulta"]')
            
            # Aguardar redirecionamento para página de produto
            page.wait_for_load_state('networkidle')
            
            # Verificar se chegou na página correta
            if '/produto' in page.url:
                print('   ✅ Redirecionado para página de resultados')
            else:
                print(f'   ⚠️ URL inesperada: {page.url}')
            
            # PASSO 4: Clicar botão "Exporta Excel"
            print('   📥 Clicando "Exporta Excel"')
            
            # Preparar para capturar o download
            with page.expect_download() as download_info:
                page.click('input[type="submit"][value="Exporta Excel"]')
            
            download = download_info.value
            
            # PASSO 5: Salvar arquivo baixado
            suggested_name = download.suggested_filename or f'cesta_basica_{data_inicial}_{data_final}.xlsx'
            
            # Garantir extensão .xlsx
            if not suggested_name.lower().endswith(('.xls', '.xlsx')):
                suggested_name += '.xlsx'
            
            # Criar nome único se arquivo já existir
            arquivo_destino = raw_data_path / suggested_name
            counter = 1
            while arquivo_destino.exists():
                name_parts = suggested_name.rsplit('.', 1)
                arquivo_destino = raw_data_path / f"{name_parts[0]}_{counter}.{name_parts[1]}"
                counter += 1
            
            # Salvar arquivo
            download.save_as(str(arquivo_destino))
            
            # Verificar se arquivo foi salvo corretamente
            if arquivo_destino.exists() and arquivo_destino.stat().st_size > 0:
                tamanho_kb = arquivo_destino.stat().st_size / 1024
                print(f'   ✅ Arquivo salvo: {arquivo_destino.name} ({tamanho_kb:.1f} KB)')
                return arquivo_destino
            else:
                print('   ❌ Arquivo não foi salvo ou está vazio')
                return None
                
        except Exception as e:
            print(f'   ❌ Erro durante coleta: {e}')
            return None
            
        finally:
            context.close()
            browser.close()

print('✅ Função de coleta criada com sucesso!')
</VSCode.Cell>

<VSCode.Cell language="python">
# Função auxiliar para validar arquivos Excel baixados
def validar_arquivo_excel(caminho: Path) -> bool:
    """Valida se o arquivo Excel foi baixado corretamente e contém dados"""
    try:
        if not caminho.exists():
            print(f'❌ Arquivo não existe: {caminho}')
            return False
            
        if caminho.stat().st_size == 0:
            print(f'❌ Arquivo vazio: {caminho}')
            return False
            
        # Tentar ler o Excel
        df = pd.read_excel(caminho, engine='openpyxl')
        
        print(f'✅ Excel válido: {caminho.name}')
        print(f'   📊 Dimensões: {len(df)} linhas × {len(df.columns)} colunas')
        print(f'   📋 Colunas: {list(df.columns)[:5]}{"..." if len(df.columns) > 5 else ""}')
        
        if len(df) > 0:
            print(f'   🔍 Primeiras linhas:')
            print(df.head(3).to_string())
            
        return True
        
    except Exception as e:
        print(f'❌ Erro ao validar Excel: {e}')
        return False

print('✅ Função de validação criada!')
</VSCode.Cell>

<VSCode.Cell language="python">
# TESTE: Coletar dados de um período curto para validar funcionamento
print('🧪 TESTE: Coletando dados de janeiro 2024')
print('=' * 50)

# Definir período de teste (janeiro 2024)
data_inicial_teste = '012024'  # janeiro 2024
data_final_teste = '012024'    # janeiro 2024

# Executar coleta de teste
arquivo_teste = coletar_cesta_basica_dieese(data_inicial_teste, data_final_teste)

if arquivo_teste:
    print('\n🎉 TESTE BEM-SUCEDIDO!')
    print(f'📄 Arquivo: {arquivo_teste.name}')
    
    # Validar arquivo baixado
    print('\n🔍 Validando arquivo...')
    if validar_arquivo_excel(arquivo_teste):
        print('✅ Arquivo válido e pronto para uso!')
    else:
        print('❌ Problema na validação do arquivo')
        
else:
    print('\n❌ TESTE FALHOU')
    print('💡 Verifique se o site está acessível ou se mudou a estrutura')

print('\n📁 Verificando arquivos na pasta raw_data:')
arquivos_existentes = list(raw_data_path.glob('*.xls*'))
for arquivo in arquivos_existentes:
    tamanho = arquivo.stat().st_size / 1024
    print(f'   📄 {arquivo.name} ({tamanho:.1f} KB)')
</VSCode.Cell>

<VSCode.Cell language="python">
# Função para coleta em lote por períodos
def coletar_lote_cesta_basica(ano_inicial: int = 2018, ano_final: int = 2025, modo: str = 'anual'):
    """
    Coleta dados em lote para múltiplos períodos
    
    Args:
        ano_inicial: ano de início (ex: 2018)
        ano_final: ano final (ex: 2025) 
        modo: 'anual' (ano completo) ou 'mensal' (mês por mês)
    """
    import json
    from datetime import datetime
    
    print(f'🚀 COLETA EM LOTE: {ano_inicial}-{ano_final} (modo: {modo})')
    print('=' * 60)
    
    resultados = []
    sucessos = 0
    falhas = 0
    
    if modo == 'anual':
        # Coletar dados anuais (janeiro a dezembro de cada ano)
        for ano in range(ano_inicial, ano_final + 1):
            data_inicial = f'01{ano}'  # janeiro
            data_final = f'12{ano}'    # dezembro
            
            print(f'\n📅 Coletando ano {ano}...')
            
            # Verificar se arquivo já existe
            arquivo_existente = list(raw_data_path.glob(f'*{data_inicial}*{data_final}*'))
            if arquivo_existente:
                print(f'   ⏭️ Pulando - arquivo já existe: {arquivo_existente[0].name}')
                resultados.append({
                    'periodo': f'{data_inicial}-{data_final}',
                    'ano': ano,
                    'status': 'existente',
                    'arquivo': str(arquivo_existente[0])
                })
                continue
            
            # Coletar dados
            arquivo = coletar_cesta_basica_dieese(data_inicial, data_final)
            
            if arquivo:
                resultados.append({
                    'periodo': f'{data_inicial}-{data_final}',
                    'ano': ano,
                    'status': 'sucesso',
                    'arquivo': str(arquivo)
                })
                sucessos += 1
                print(f'   ✅ Ano {ano} coletado com sucesso')
            else:
                resultados.append({
                    'periodo': f'{data_inicial}-{data_final}',
                    'ano': ano,
                    'status': 'falha',
                    'arquivo': None
                })
                falhas += 1
                print(f'   ❌ Falha na coleta do ano {ano}')
                
            # Pausa entre requisições
            import time
            time.sleep(2)
    
    elif modo == 'mensal':
        # Coletar dados mensais
        for ano in range(ano_inicial, ano_final + 1):
            for mes in range(1, 13):
                data_inicial = f'{mes:02d}{ano}'
                data_final = f'{mes:02d}{ano}'
                
                print(f'\n📅 Coletando {mes:02d}/{ano}...')
                
                # Verificar se arquivo já existe
                arquivo_existente = list(raw_data_path.glob(f'*{data_inicial}*{data_final}*'))
                if arquivo_existente:
                    print(f'   ⏭️ Pulando - arquivo já existe')
                    continue
                
                # Coletar dados
                arquivo = coletar_cesta_basica_dieese(data_inicial, data_final)
                
                if arquivo:
                    sucessos += 1
                    print(f'   ✅ {mes:02d}/{ano} coletado')
                else:
                    falhas += 1
                    print(f'   ❌ Falha {mes:02d}/{ano}')
                
                time.sleep(1)  # Pausa menor para coletas mensais
    
    # Salvar metadata da coleta
    metadata = {
        'data_coleta': datetime.now().isoformat(),
        'modo': modo,
        'periodo': f'{ano_inicial}-{ano_final}',
        'total_sucessos': sucessos,
        'total_falhas': falhas,
        'resultados': resultados
    }
    
    arquivo_metadata = metadata_path / f'coleta_{modo}_{ano_inicial}_{ano_final}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
    
    with open(arquivo_metadata, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    
    # Relatório final
    print(f'\n📋 RELATÓRIO FINAL DA COLETA EM LOTE:')
    print(f'   ✅ Sucessos: {sucessos}')
    print(f'   ❌ Falhas: {falhas}')
    print(f'   📊 Taxa de sucesso: {(sucessos/(sucessos+falhas)*100) if (sucessos+falhas) > 0 else 0:.1f}%')
    print(f'   💾 Metadata salva em: {arquivo_metadata.name}')
    
    return metadata

print('✅ Função de coleta em lote criada!')
</VSCode.Cell>

<VSCode.Cell language="python">
# EXECUÇÃO: Coleta completa dos dados históricos (2018-2025)
print('🎯 INICIANDO COLETA COMPLETA DE DADOS HISTÓRICOS')
print('📅 Período: 2018-2025 (dados anuais)')
print('⚠️ Esta operação pode demorar alguns minutos...')
print()

# Confirmar execução
resposta = input('🤔 Prosseguir com a coleta completa? (s/n): ')

if resposta.lower() == 's':
    print('\n🚀 Iniciando coleta completa...')
    
    # Executar coleta anual (janeiro a dezembro de cada ano)
    metadata_coleta = coletar_lote_cesta_basica(
        ano_inicial=2018, 
        ano_final=2025, 
        modo='anual'
    )
    
    print('\n🎉 COLETA COMPLETA FINALIZADA!')
    
    # Verificar arquivos coletados
    arquivos_coletados = list(raw_data_path.glob('*.xls*'))
    arquivos_coletados.sort()
    
    print(f'\n📁 ARQUIVOS COLETADOS ({len(arquivos_coletados)}):')
    tamanho_total = 0
    
    for arquivo in arquivos_coletados:
        tamanho = arquivo.stat().st_size / 1024
        tamanho_total += tamanho
        print(f'   📄 {arquivo.name} ({tamanho:.1f} KB)')
    
    print(f'\n💾 Tamanho total: {tamanho_total/1024:.1f} MB')
    print(f'📂 Localização: {raw_data_path}')
    
else:
    print('⏹️ Coleta cancelada pelo usuário')
    print('💡 Execute a célula novamente quando quiser fazer a coleta completa')
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Resumo e Próximos Passos

### ✅ Status da Coleta Automatizada

**COLETA IMPLEMENTADA:** Sistema funcional seguindo o fluxo correto do DIEESE

**Funcionalidades:**
- 🎯 **Fluxo correto:** Preenche formulário → Consulta → Download Excel
- 🤖 **Playwright/Chromium:** Navegador empacotado no venv (sem sudo)
- 📅 **Períodos flexíveis:** Coleta anual ou mensal conforme necessário
- 💾 **Organização:** Arquivos salvos em `raw_data/` com metadata
- ✅ **Validação:** Verificação automática dos arquivos baixados

**Dados coletados:**
- **Fonte:** DIEESE - Departamento Intersindical de Estatística
- **Tipo:** Cesta básica nacional por capital
- **Período:** 2018-2025 (conforme disponibilidade)
- **Formato:** Excel (.xlsx) prontos para processamento

### 🚀 Próximos Passos

1. **Processamento:** Limpar e estruturar os dados Excel coletados
2. **Feature Engineering:** Criar indicadores regionais e temporais  
3. **Integração:** Unir com dados de combustíveis e outros indicadores macro
4. **Modelagem:** Desenvolver modelo de risco de crédito

### 📊 Estrutura Criada

```
data/external_data/macro_specified_data/cesta_basica/
├── raw_data/           # ✅ Planilhas Excel do DIEESE
├── processed_data/     # 🔄 Para próxima etapa
└── metadata/          # ✅ Logs de coleta automatizada
```

---

*Este notebook resolve definitivamente a coleta de dados do DIEESE seguindo o fluxo correto do site.*
</VSCode.Cell>
````